<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/ons_study_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# @title [CELL 1] Setup + Helper Functions (V106 - Nav Reference Links)
import os
import sys
import re
import csv
import json
import html
import socket
import pytz
import pandas as pd
from ftplib import FTP_TLS
from datetime import datetime

# --- INSTALL TQDM IF MISSING ---
try:
    import tqdm
except ImportError:
    os.system('pip install tqdm')
    import tqdm

print("="*60)
print("      [CELL 1] SETUP LOADED (V106)")
print("      (Includes: Cousin Calc & cM Chart Links)")
print("="*60)

# ==============================================================================
# 1. GLOBAL HELPER FUNCTIONS + HTML ASSETS
# ==============================================================================
TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

NAV_HTML = """
<style>
nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none}
nav.oldnav li{display:inline-block}
nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px}
nav.oldnav a:hover{background-color:#00838f!important}
@media print { nav.oldnav, #nav-slot, .no-print, .action-btn, .control-panel, .tabs { display: none !important; } }
</style>
<nav class="oldnav"><ul>
<li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li>
<li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li>
<li><a href="/ons-study/yates_ancestor_register.shtml">DNA Register</a></li>
<li><a href="/ons-study/just-trees.shtml">Trees</a></li>
<li><a href="/ons-study/dna_network.shtml">DNA Network</a></li>
<li><a href="/ons-study/lineage_proof.html">Lineage Proof</a></li>
<li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li>
<li><a href="/ons-study/brick_wall_buster.shtml" style="background:#f57f17;color:black !important;">Brick Wall Buster</a></li>
<li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li>
<li><a href="https://yates.one-name.net/gengen/images/cousin-calculator.jpg" target="_blank" style="color:#b2dfdb;">Cousin Calc</a></li>
<li><a href="https://yates.one-name.net/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" style="color:#b2dfdb;">cM Chart</a></li>
<li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li>
<li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li>
</ul></nav>"""

SITE_INFO = """<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;font-size:1.05em;margin-bottom:0;"><strong>Methodology:</strong> This register moves beyond the reliance on single "golden matches" to prove kinship. Instead, it employs <em>Collateral DNA Saturation</em>—a method that blends genealogical reasoning with data-driven logic.</p></div>"""

# V86: Added Table Filtering Logic
JS_CORE = r"""<script type="text/javascript">
(function(){
    // SORTING LOGIC
    function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/\s+/g,' ').trim().toLowerCase();}
    function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.\-]/g,'')),nB=parseFloat(B.replace(/[^0-9.\-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);}
    function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(/\s+\(asc\)|\s+\(desc\)/,'');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}}

    // FILTERING LOGIC
    window.filterTable = function() {
        var input = document.getElementById("tableSearch");
        var filter = input.value.toUpperCase();
        var table = document.getElementById("reg-table") || document.querySelector("table.dataframe");
        var tr = table.getElementsByTagName("tr");
        for (var i = 1; i < tr.length; i++) {
            var tdArr = tr[i].getElementsByTagName("td");
            var found = false;
            for (var j = 0; j < tdArr.length; j++) {
                if (tdArr[j]) {
                    var txtValue = tdArr[j].textContent || tdArr[j].innerText;
                    if (txtValue.toUpperCase().indexOf(filter) > -1) {
                        found = true;
                        break;
                    }
                }
            }
            tr[i].style.display = found ? "" : "none";
        }
    }

    function init(){
        var t=document.getElementsByTagName('table');
        for(var i=0;i<t.length;i++) if(/\bsortable\b/.test(t[i].className)) makeSortable(t[i]);
    }
    if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init();
})();
</script>"""

GLOSSARY_CONTENT = """<div style="background:white;padding:25px;border-radius:8px;border:1px solid #ddd;font-family:sans-serif;line-height:1.6;"><h2 style="color:#006064;border-bottom:2px solid #004d40;padding-bottom:10px;">ONS Yates Study: Data Glossary</h2><h3 style="color:#00838f;margin-top:25px;">1. Identity Columns</h3><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Tester-Participant-MASKED (The Trigger):</strong><br>The unique privacy code extracted from the user's NPFX tag.</li><li style="margin-bottom:15px;"><strong>Tester-Participant-Unmasked:</strong><br>The real name of the tester.</li></ul><h3 style="color:#00838f;margin-top:25px;">2. Analysis Terms</h3><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Platinum Standard:</strong><br>Lineages with 30+ matches and 10+ unique sources. Biologically proven.</li><li style="margin-bottom:15px;"><strong>Keystone Tester:</strong><br>A high-value participant (15+ matches) who anchors a specific branch.</li><li style="margin-bottom:15px;"><strong>Forensic Handshake:</strong><br>An informal term in genetic genealogy describing a confirmation pattern in which multiple independent DNA matches support the same ancestral line or family connection, strengthening confidence in an identification or relationship conclusion. Rather than relying on a single match, genealogists look for several matches that converge on the same family network, creating a “handshake” of agreement between genetic evidence and documentary research. This concept is especially valuable in unknown-parentage and forensic cases, where conclusions must be supported by redundant evidence.<br><br>A forensic handshake is often achieved through related methods such as triangulation, where three or more individuals share the same DNA segment from a common ancestor, and cluster analysis, which groups matches who also match one another and often represent a shared ancestral couple or lineage. Together, these approaches help meet an emerging genetic genealogy proof standard, emphasizing that reliable conclusions require multiple corroborating matches, consistency with records, and the exclusion of alternate explanations.</li></ul><h3 style="color:#00838f;margin-top:25px;">3. Genealogy Concepts</h3><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Spanish Naming System:</strong><br>A traditional Hispanic naming convention in which an individual bears one or more given names followed by two surnames: the first inherited from the father (paternal surname) and the second from the mother (maternal surname). This system is historically rooted in Spain and is especially useful in genealogy because it preserves both parental lineages and improves identification in historical records.</li><li style="margin-bottom:15px;"><strong>Née:</strong><br>A term meaning “born as,” used to indicate a woman’s maiden or birth surname before marriage. In genealogical and historical records, née identifies the surname a woman carried in her natal family line, preserving her connection to her parents and ancestry. For example, “Maria García, née López” shows that María’s birth surname was López, even though she later used García after marriage.<br><br>The use of née is especially important for tracing maternal family lines, since women’s surnames may change across generations in many cultures. By recording a woman’s birth name, genealogists can correctly link her to her original family, distinguish between individuals with similar married names, and maintain continuity in lineage reconstruction, particularly in marriage, probate, and church records.</li></ul></div>"""

SUBSCRIBE_CONTENT = """<div style="background:white;padding:40px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);max-width:800px;margin:30px auto;text-align:center;font-family:'Segoe UI',sans-serif;"><h1 style="color:#006064;margin-bottom:10px;">Join the Yates Research Community</h1><p style="color:#555;font-size:1.1em;line-height:1.6;margin-bottom:30px;">Stay connected with the latest breakthroughs in the Yates One-Name Study. Get notified about new DNA groups, lineage verifications, and quarterly reports.</p><div style="background:#e0f2f1;padding:25px;border-radius:8px;border:1px solid #b2dfdb;display:inline-block;"><h3 style="margin-top:0;color:#004d40;">📧 One-Click Subscribe</h3><p style="margin-bottom:20px;">Click below to send a subscription request to our Groups.io list.</p><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block;padding:15px 30px;background:#00838f;color:white;text-decoration:none;border-radius:5px;font-weight:bold;font-size:1.1em;box-shadow:0 2px 5px rgba(0,0,0,0.2);">Subscribe Now</a></div><p style="margin-top:30px;font-size:0.9em;color:#777;">Powered by Groups.io. You will receive a confirmation email shortly.</p></div>"""

SHARE_CONTENT = """<div style="max-width:900px;margin:30px auto;font-family:'Segoe UI',sans-serif;line-height:1.6;color:#333;"><div style="text-align:center;margin-bottom:40px;"><h1 style="color:#0277bd;margin-bottom:10px;">Share Your Ancestry DNA Matches</h1><p style="font-size:1.1em;color:#555;">Ancestry provides a built-in sharing feature that allows you to grant limited access to your DNA matches <strong>without sharing your personal account details</strong>. You remain in full control of your account at all times.</p></div><div style="display:grid;grid-template-columns:1fr 1fr;gap:30px;margin-bottom:30px;"><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #0277bd;"><h3 style="color:#0277bd;margin-top:0;">How Sharing Works</h3><p>From your AncestryDNA <strong>Settings</strong> page, you may invite another individual by email and assign one of the following roles:</p><ul style="padding-left:20px;"><li><strong>Viewer</strong> (Read only)</li><li><strong>Collaborator</strong> (Recommended for Study)</li><li><strong>Manager</strong> (Full Control)</li></ul></div><div style="background:#e3f2fd;padding:25px;border-radius:8px;border:1px solid #90caf9;"><h3 style="color:#01579b;margin-top:0;">Privacy & Control</h3><p>This sharing arrangement provides <strong>limited access only</strong>. It does not allow changes to your account and does not expose your personal details.</p><p><strong>You may revoke access at any time through Ancestry.</strong></p></div></div><div style="background:white;padding:30px;border-radius:8px;border:1px solid #ddd;box-shadow:0 4px 15px rgba(0,0,0,0.05);"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">How to Share for the Yates One-Name Study</h2><ol style="font-size:1.1em;line-height:1.8;padding-left:25px;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Scroll to the section labeled "DNA Result Sharing" and click <strong>Invite</strong>.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="background:#fff3e0;padding:5px 10px;border-radius:4px;font-weight:bold;color:#e65100;font-family:monospace;font-size:1.2em;">yatesvilleron@gmail.com</span></li><li>Assign the role: <strong>Collaborator</strong>.</li></ol><div style="background:#fffde7;border-left:5px solid #fbc02d;padding:15px;margin-top:20px;font-size:0.95em;"><strong>Why Collaborator?</strong> The Collaborator role allows Ron to review shared matches and create small internal groups (colored dots) to identify which matches have been reviewed and which have contributed evidence to the Yates One-Name Study.</div></div><div style="margin-top:40px;"><h3 style="color:#006064;">What Happens Next?</h3><p>After sharing, you will receive an invitation to subscribe to the <strong>Yates One-Name Study Groups.io mailing list</strong>, where DNA proof summaries and study findings are shared.</p><h3 style="color:#006064;">Reciprocal Sharing (Optional)</h3><p>If you are interested in viewing Ron’s DNA matches, simply let him know. When a direct match exists, that relationship will be reflected in the study findings.</p></div></div>"""

THEORY_CONTENT = """<div style="max-width:1000px;margin:30px auto;font-family:'Segoe UI',sans-serif;line-height:1.6;color:#333;"><h1 style="text-align:center;color:#004d40;font-size:2.5em;margin-bottom:10px;">The Yates DNA Strategy</h1><p style="text-align:center;font-size:1.2em;color:#666;margin-bottom:40px;">Moving beyond traditional Y-DNA to solve modern genealogical mysteries.</p><div style="display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:30px;margin-bottom:40px;"><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #00838f;"><h2 style="color:#006064;margin-top:0;">The Autosomal Revolution</h2><p>Traditional one-name studies rely almost exclusively on Y-DNA to trace the paternal surname line. While valuable for deep history, this approach ignores 50% of our ancestors at every generation.</p><p><strong>Our Focus:</strong> We utilize <strong>Autosomal DNA (atDNA)</strong>—inherited from both parents—to verify connections across <em>all</em> branches. This allows us to:</p><ul style="padding-left:20px;color:#444;"><li>Bridge the "Gender Gap" by tracing female descendants.</li><li>Verify paper trails for the last 300 years (Genealogical Time).</li><li>Cluster "Floating" Yates families into their correct lines.</li></ul></div><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #f9a825;"><h2 style="color:#f57f17;margin-top:0;">Collateral Saturation</h2><p>A single DNA match can be luck. Ten matches is a statistic. <strong>Thirty matches is a fact.</strong></p><p>We employ a technique called <strong>Collateral Saturation</strong>. We don't look for one "Golden Match." We analyze groups of matches from independent cousin lines. When descendants from four different children of <em>William & Mary Yates</em> all share DNA with you, the relationship is biologically confirmed.</p><div style="text-align:center;margin-top:20px;"><a href="dna_network.shtml" style="display:inline-block;padding:10px 20px;background:#f9a825;color:#333;text-decoration:none;border-radius:4px;font-weight:bold;">View the Network</a></div></div></div><div style="background:#e0f2f1;padding:30px;border-radius:8px;border:1px solid #b2dfdb;margin-bottom:40px;"><h2 style="color:#004d40;margin-top:0;text-align:center;">From Theory to Tools</h2><p style="text-align:center;max-width:700px;margin:0 auto 20px auto;">We have built a suite of forensic tools to visualize this data. Instead of raw spreadsheets, we offer interactive dashboards to prove your connection.</p><div style="display:flex;flex-wrap:wrap;justify-content:center;gap:15px;margin-top:20px;"><a href="ons_yates_dna_register.shtml" style="background:#006064;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">The Register</a><a href="lineage_proof.html" style="background:#00838f;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">Proof Engine</a><a href="dna_dossier.html" style="background:#00acc1;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">Forensic Dossier</a></div></div><div style="background:#f5f5f5;padding:20px;border-radius:8px;border:1px solid #ddd;"><h3 style="color:#555;margin-top:0;">Legacy Data: Y-DNA Haplogroups</h3><p style="font-size:0.9em;color:#666;margin-bottom:15px;">Y-DNA is the backbone of deep ancestry (27,000 BCE to 1600 AD). While not our primary focus for recent genealogy, we maintain a detailed record of the Yates Y-Chromosome mutations (R-M207 through FT266579).</p><a href="https://yates.one-name.net/gengen/dna_proof_y.htm" style="color:#006064;font-weight:bold;text-decoration:none;">&raquo; View Detailed Y-DNA Findings</a></div></div>"""

def make_page(title, content, count, view_type="", extra="", stats_bar=""):
    nav_blk = ""
    if view_type in ['ancestor', 'participant', 'tree_az', 'tree_za', 'proof', 'network', 'dossier', 'subscribe', 'share', 'buster', 'singleton']:
        nav_blk = SITE_INFO
    if view_type == 'subscribe' or view_type == 'theory' or view_type == 'share':
        nav_blk = ""

    toggle = ""
    print_btn = ""
    search_bar = ""

    # V86: Add Search Bar for Registers and Singletons
    if view_type in ['ancestor', 'participant', 'singleton']:
        search_bar = """<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div>"""

    if view_type in ['ancestor', 'participant', 'singleton']:
        view_name = "Register"
        if view_type == 'singleton': view_name = "Singleton List"
        print_btn = f"""<div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print {view_name}</button></div>"""

    if view_type == 'ancestor':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="font-weight:bold;color:#006064;">By Ancestral Line</a> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1;text-decoration:none;">By Participant Name</a></div>"""
    elif view_type == 'participant':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1;text-decoration:none;">By Ancestral Line</a> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="font-weight:bold;color:#006064;">By Participant Name</a></div>"""
    elif 'tree' in view_type:
        za = f'<span style="font-weight:bold;color:#000;">Z-A</span>' if 'za' in view_type else f'<a href="just-trees.shtml" style="color:#006064;text-decoration:underline;">Z-A</a>'
        az = f'<span style="font-weight:bold;color:#000;">A-Z</span>' if 'az' in view_type else f'<a href="just-trees-az.shtml" style="color:#006064;text-decoration:underline;">A-Z</a>'
        toggle = f"""<div class="no-print" style="text-align:center;font-family:sans-serif;font-size:16px;margin:15px 0 10px 0;">Individual Yates Family trees: &nbsp; {za} &nbsp;|&nbsp; {az}</div>"""

    return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css">{extra}</head><body id="top"><div class="wrap"><h1 class="centerline">{title}</h1><div id="nav-slot">{stats_bar}{NAV_HTML}</div>{nav_blk}{search_bar}{print_btn}{toggle}{content}{JS_CORE}</div></body></html>"""

print("✅ Cell 1 Loaded! (Includes Cousin Calc & cM Chart Links)")

      [CELL 1] SETUP LOADED (V106)
      (Includes: Cousin Calc & cM Chart Links)
✅ Cell 1 Loaded! (Includes Cousin Calc & cM Chart Links)


In [51]:
# @title [CELL 3] The Data Engine (V123.2 - Deep-Space Probe)
def run_engine():
    print("="*60)
    print("      [CELL 3] ENGINE STARTING (V123.2 - DEEP SPACE PROBE)...")
    print("="*60)

    import os
    import sys
    import re
    import csv
    from ftplib import FTP_TLS
    from google.colab import userdata
    from datetime import datetime

    # --- 1. FORCE CLEANUP ---
    CSV_DB = "engine_database.csv"
    if os.path.exists(CSV_DB):
        os.remove(CSV_DB)
        print("    🗑️ Deleted old database (Forcing fresh build)")

    # --- CONFIGURATION ---
    HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
    USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
    PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

    REMOTE_SUBDIR = "ons-study"
    KEY_FILE       = "match_to_unmasked.csv"
    PROCESSED_GED  = "_processed_unmasked.ged"

    # --- HELPER: CLEANING ---
    def clean_and_standardize(raw_name):
        if not raw_name: return "findme"
        s = raw_name.replace("/", "").strip()
        triggers = ["unknown", "missing", "searching", "still searching", "living", "private", "nee", "nee ?", "wife", "mrs"]
        if s.lower() in triggers or s == "": return "findme"
        if "?" in s: return "findme"
        if "unknown" in s.lower(): return "findme"
        return s

    def get_surname(full_name):
        if not full_name or "findme" in full_name.lower(): return ""
        clean = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', full_name, flags=re.IGNORECASE)
        parts = clean.replace(',', '').split()
        return parts[-1] if parts else ""

    def make_directory_label(name, dates):
        if "findme" in name.lower(): return name
        sur = get_surname(name)
        if not sur: return name
        firsts = re.sub(f"{re.escape(sur)}$", "", name).strip()
        return f"{sur}, {firsts} {dates}"

    def make_cojoined_key(full_name):
        if not full_name or "findme" in full_name.lower(): return "zzz"
        clean = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', full_name, flags=re.IGNORECASE)
        clean = re.sub(r'[\,\.]', '', clean)
        parts = clean.split()
        if not parts: return "zzz"
        if len(parts) == 1: return parts[0]
        surname = parts[-1]
        firsts = "".join(parts[:-1])
        return f"{surname}{firsts}"

    # ---------------------------------------------------------
    # STEP 1 & 2: SETUP
    # ---------------------------------------------------------
    print("\n[STEP 1] Setup...")
    try:
        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()
        try:
            with open(KEY_FILE, "wb") as f: ftps.retrbinary(f"RETR /{REMOTE_SUBDIR}/{KEY_FILE}", f.write)
        except: pass
        ftps.quit()
    except: pass

    all_files = os.listdir('.')
    ged_files = [f for f in all_files if f.lower().endswith('.ged') and "_processed" not in f and "unmasked" not in f.lower()]
    if not ged_files: return print("❌ No GEDCOM found.")
    ged_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    DEFAULT_GEDCOM = ged_files[0]
    print(f"    👉 Source: {DEFAULT_GEDCOM}")

    # ---------------------------------------------------------
    # STEP 3: UNMASKING
    # ---------------------------------------------------------
    unmask_map = {}
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, 'r', errors='replace') as f:
            for row in csv.reader(f):
                if len(row)>1: unmask_map[row[0].strip().lower()] = row[1].strip()

    def resolve_code(payload):
        m = re.search(r'(\d+)\s*&?\s*([^ \t\n\r\f\v]+)', payload)
        return m.group(2).lower() if m else None

    with open(DEFAULT_GEDCOM, 'r', encoding='utf-8', errors='replace') as fin, \
         open(PROCESSED_GED, 'w', encoding='utf-8') as fout:
        buffer = []; real_name = None
        for line in fin:
            if line.startswith("0 @"):
                if buffer:
                    for b in buffer:
                        if b.startswith("1 NAME") and real_name: fout.write(f"1 NAME {real_name}\n")
                        else: fout.write(b)
                buffer = [line]; real_name = None
            else:
                buffer.append(line)
                if line.startswith("1 NPFX"):
                    parts = line.split(" ", 2)
                    if len(parts) > 2:
                        code = resolve_code(parts[2].strip())
                        if code: real_name = unmask_map.get(code, code)
        if buffer:
            for b in buffer:
                if b.startswith("1 NAME") and real_name: fout.write(f"1 NAME {real_name}\n")
                else: fout.write(b)

    # ---------------------------------------------------------
    # STEP 4: TRACE
    # ---------------------------------------------------------
    print("\n[STEP 4] Tracing Lineages...")
    individuals = {}; families = {}

    def is_yates(name_str):
        n = (name_str or "").lower()
        return "yates" in n or "yeates" in n or "yate" in n

    current_id = None; current_fam = None; current_tag = None
    with open(PROCESSED_GED, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip(); parts = line.split(" ", 2)
            if len(parts) < 2: continue
            lvl, tag, val = parts[0], parts[1], parts[2] if len(parts)>2 else ""

            if lvl == "0" and "INDI" in val:
                current_id = tag.replace("@", "")
                individuals[current_id] = {"name": "findme", "famc": None, "fams": [], "code": "", "cm": 0, "birt": "", "deat": ""}
                current_fam = None; current_tag = "INDI"
            elif current_id and lvl != "0":
                if tag == "NAME": individuals[current_id]["name"] = clean_and_standardize(val)
                elif tag == "FAMC": individuals[current_id]["famc"] = val.replace("@", "")
                elif tag == "FAMS": individuals[current_id]["fams"].append(val.replace("@", ""))
                elif tag == "NPFX":
                    code = resolve_code(val)
                    if code: individuals[current_id]["code"] = code
                    m = re.search(r'^(\d+)|(\d+)\s*cM', val, re.IGNORECASE)
                    if m: individuals[current_id]["cm"] = int(m.group(1) or m.group(2))
                elif tag == "BIRT": current_tag = "BIRT"
                elif tag == "DEAT": current_tag = "DEAT"
                elif tag == "DATE" and current_tag:
                    m = re.search(r'\d{4}', val)
                    if m: individuals[current_id][current_tag.lower()] = m.group(0)
                    current_tag = None

            if lvl == "0" and "FAM" in val:
                current_fam = tag.replace("@", "")
                families[current_fam] = {"husb": None, "wife": None}
                current_id = None
            elif current_fam and lvl != "0":
                if tag == "HUSB": families[current_fam]["husb"] = val.replace("@", "")
                elif tag == "WIFE": families[current_fam]["wife"] = val.replace("@", "")

    def get_parents(pid):
        if not pid or pid not in individuals: return None, None
        famc = individuals[pid]["famc"]
        if not famc or famc not in families: return None, None
        return families[famc]["husb"], families[famc]["wife"]

    def get_mother_surname(pid):
        if not pid: return ""
        _, mom_id = get_parents(pid)
        if mom_id and mom_id in individuals:
            return get_surname(individuals[mom_id]["name"])
        return ""

    def to_spanish_name(pid, current_name):
        if "findme" in current_name.lower(): return current_name
        mom_surname = get_mother_surname(pid)
        if not mom_surname or "findme" in mom_surname.lower(): return current_name
        if mom_surname.lower() not in current_name.lower():
            return f"{current_name}-{mom_surname}"
        return current_name

    # --- V123.2: DEEP SPACE PROBE (25 Generations) ---
    def probe_for_yates(pid, depth=25):
        if depth <= 0 or not pid or pid not in individuals: return False

        name = individuals[pid]["name"]
        if is_yates(name): return True

        dad_id, mom_id = get_parents(pid)
        # Probe BOTH lines aggressively
        if dad_id and probe_for_yates(dad_id, depth-1): return True
        if mom_id and probe_for_yates(mom_id, depth-1): return True

        return False

    def climb_full_line(start_id):
        curr = start_id; lineage_data = []
        prev_famc = individuals[start_id]["famc"]

        while curr:
            p = individuals.get(curr)
            if not p: break

            spanish_name = to_spanish_name(curr, p["name"])

            # --- SPOUSE MATCHING ---
            spouse_name = "findme"; spouse_id = None
            active_fam_id = None
            if prev_famc and prev_famc in p["fams"]: active_fam_id = prev_famc
            elif p["fams"]: active_fam_id = p["fams"][0]

            if active_fam_id and active_fam_id in families:
                f = families[active_fam_id]
                sid = f["wife"] if f["husb"] == curr else f["husb"]
                if sid and sid in individuals:
                    spouse_name = individuals[sid]["name"]
                    spouse_id = sid

            spouse_spanish = to_spanish_name(spouse_id, spouse_name) if spouse_id else spouse_name

            lineage_data.append({
                "name": spanish_name,
                "raw_name": p["name"],
                "id": curr,
                "spouse": spouse_spanish,
                "spouse_raw": spouse_name,
                "spouse_id": spouse_id
            })

            # --- DECISION: WHO TO CLIMB? ---
            dad_id, mom_id = get_parents(curr)
            prev_famc = p["famc"]

            if not dad_id and not mom_id: break

            dad_n = individuals.get(dad_id, {}).get("name", "findme") if dad_id else "findme"
            mom_n = individuals.get(mom_id, {}).get("name", "findme") if mom_id else "findme"

            # 1. Immediate Check
            dad_is_yates = is_yates(dad_n)
            mom_is_yates = is_yates(mom_n)

            if dad_is_yates and not mom_is_yates:
                curr = dad_id
            elif mom_is_yates and not dad_is_yates:
                curr = mom_id
            else:
                # 2. DEEP PROBE (Extended to 25 Gens)
                dad_leads = probe_for_yates(dad_id)
                mom_leads = probe_for_yates(mom_id)

                if mom_leads and not dad_leads:
                    curr = mom_id
                elif dad_leads:
                    curr = dad_id
                else:
                    curr = dad_id # Blind default if neither leads to Yates

        return lineage_data

    def format_dates(uid):
        if not uid or uid not in individuals: return "findme"
        b = individuals[uid]["birt"] or "findme"
        d = individuals[uid]["deat"] or "findme"
        b = re.sub(r'\?', 'findme', b)
        d = re.sub(r'\?', 'findme', d)
        if b == "findme" and d == "findme": return "findme"
        return f"({b} - {d})"

    rows = []
    for uid, p in individuals.items():
        if p["code"]: # Tester
            lineage_data = climb_full_line(uid)
            if not lineage_data: continue

            full_line = list(reversed(lineage_data))
            gen1 = full_line[0]

            top_name = gen1["raw_name"]
            top_dates = format_dates(gen1["id"])
            spouse_name = gen1["spouse_raw"]
            spouse_id = gen1["spouse_id"]
            spouse_dates = format_dates(spouse_id)

            if spouse_name != "findme":
                husb_sur = get_surname(top_name)
                wife_sur = get_surname(spouse_name)
                if husb_sur.lower() == wife_sur.lower():
                    spouse_name += f" (née {wife_sur})"

            pair_dated = f"{top_name} {top_dates}"
            if spouse_name != "findme": dir_label = make_directory_label(top_name, top_dates) + f" & {spouse_name}"
            else: dir_label = make_directory_label(top_name, top_dates)

            if spouse_name != "findme": pair_dated += f" & {spouse_name} {spouse_dates}"
            pair_simple = f"{top_name} & {spouse_name}" if spouse_name != "findme" else top_name

            clean_top = re.sub(r'[^a-zA-Z0-9]', '', top_name)
            clean_sp = re.sub(r'[^a-zA-Z0-9]', '', spouse_name.split('(')[0]) if spouse_name != "findme" else "ZZZ"
            sort_key = f"{clean_top}_{clean_sp}"

            path_names = []
            search_ids_list = []
            search_names_list = []

            for i, x in enumerate(full_line):
                if i == 0: path_names.append(pair_dated)
                else: path_names.append(x["name"])

                if x["id"]:
                    search_ids_list.append(x["id"])
                    search_names_list.append(x["name"])
                if x["spouse_id"]:
                    search_ids_list.append(x["spouse_id"])
                    search_names_list.append(x["spouse_raw"])

            lineage_str = " -> ".join(path_names)
            path_ids = ",".join([x["id"] for x in full_line])
            search_inventory_ids = ",".join(search_ids_list)
            search_inventory_names = "|".join(search_names_list)

            _, fa1_mom_id = get_parents(gen1["id"])
            fa1_mother = to_spanish_name(fa1_mom_id, individuals[fa1_mom_id]["name"]) if fa1_mom_id else "findme"

            fa2_mother = "findme"
            if spouse_id:
                _, fa2_mom_id = get_parents(spouse_id)
                if fa2_mom_id: fa2_mother = to_spanish_name(fa2_mom_id, individuals[fa2_mom_id]["name"])

            unmasked_name = unmask_map.get(p["code"], p["code"])
            cojoined_key = make_cojoined_key(unmasked_name)

            rows.append({
                "Tester-Participant-MASKED": p["code"],
                "Tester-Participant-Unmasked": unmasked_name,
                "tester-cojoin-lastfirst": cojoined_key,
                "Found Match": p["name"],
                "ID#": uid, "cM": p["cm"], "Spacer": "",
                "Yates DNA Ancestral Line": lineage_str,
                "Authority_Directory_Label": dir_label,
                "Authority_FirstAncestor": pair_simple,
                "Authority_FirstAncestor_alpha": sort_key,
                "Authority_FirstAncestor_dated": pair_dated,
                "fa_1 extracted": top_name, "fa_1_Dates": top_dates, "fa_1_Mother": fa1_mother,
                "fa_2 extracted": spouse_name, "fa_2 Dates": spouse_dates, "fa_2_Mother": fa2_mother,
                "Gen_Count": len(full_line),
                "Ancestral_Path_IDs": path_ids,
                "Search_Inventory_IDs": search_inventory_ids,
                "Search_Inventory_Names": search_inventory_names
            })

    rows.sort(key=lambda r: r["Authority_Directory_Label"])

    fieldnames = [
        "Tester-Participant-MASKED", "Tester-Participant-Unmasked", "tester-cojoin-lastfirst",
        "Found Match", "ID#", "cM", "Spacer",
        "Yates DNA Ancestral Line", "Authority_Directory_Label",
        "Authority_FirstAncestor", "Authority_FirstAncestor_alpha", "Authority_FirstAncestor_dated",
        "fa_1 extracted", "fa_1_Dates", "fa_1_Mother",
        "fa_2 extracted", "fa_2 Dates", "fa_2_Mother",
        "Gen_Count", "Ancestral_Path_IDs", "Search_Inventory_IDs", "Search_Inventory_Names"
    ]

    with open(CSV_DB, "w", encoding="iso-8859-15", newline="", errors="replace") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader(); writer.writerows(rows)

    print(f"\n[SUCCESS] Engine V123.2 Complete. Saved {len(rows)} verified matches to {CSV_DB} (Deep Space Probe Active).")

print("✅ Cell 3 (Engine V123.2) Loaded.")

✅ Cell 3 (Engine V123.2) Loaded.


In [52]:
# @title [CELL 4] The Publisher (V122 - Robust Search)
def run_publisher():
    print("="*60)
    print("      [CELL 4] PUBLISHER STARTING (V122 - ROBUST SEARCH)...")
    print("="*60)

    import os
    import sys
    import json
    import pytz
    import pandas as pd
    from datetime import datetime
    import re
    from google.colab import userdata

    # --- CONFIGURATION ---
    REMOTE_SUBDIR = "ons-study"

    # --- HELPER FUNCTIONS ---
    TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
    TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

    def normalize_id(val):
        s = str(val).replace('@', '').strip()
        if s.isdigit(): return f"I{s}"
        return s

    def build_narrative(row):
        part_name = str(row.get('Tester-Participant-Unmasked', '')).strip()
        cm = str(row.get('cM', '0'))
        anc_dated = str(row.get('Authority_FirstAncestor_dated', 'Unknown'))
        found_match = str(row.get('Found Match', 'Unknown')).strip()
        gen_count = row.get('Gen_Count', 0)
        rid = normalize_id(row.get('ID#', ''))
        linked_found_match = f'<a href="{TNG_BASE_URL}{rid}{TNG_SUFFIX}" target="_blank"><b>{found_match}</b></a>'
        return f"{part_name} is a {cm} cM DNA match to {linked_found_match} is related via {anc_dated} back {gen_count} generations."

    def build_linked_lineage(row):
        line = str(row.get('Yates DNA Ancestral Line', ''))
        found = str(row.get('Found Match', ''))
        rid = normalize_id(row.get('ID#', ''))
        if found and rid and found in line:
            url = f"{TNG_BASE_URL}{rid}{TNG_SUFFIX}"
            link_html = f'<a href="{url}" target="_blank" style="color:#006064;text-decoration:none;font-weight:bold;">{found}</a>'
            return line.replace(found, link_html)
        return line

    def get_sortable_surname(full_name):
        if not full_name: return "zzz"
        s = str(full_name).lower()
        s = re.split(r'\bnee\b|\bnée\b', s)[0]
        s = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', s)
        s = re.sub(r'[\,\.]', '', s)
        parts = s.strip().split()
        if not parts: return "zzz"
        if len(parts) >= 3 and parts[-2] == 'y': return parts[-3]
        return parts[-1]

    def make_participant_key(full_name):
        if not full_name: return "zzz"
        sur = get_sortable_surname(full_name)
        s_clean = full_name.lower().replace(sur, "").strip()
        return f"{sur}, {s_clean}".title()

    def format_last_first(full_name):
        if not full_name: return ""
        s = str(full_name)
        cleaned = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', s, flags=re.IGNORECASE)
        cleaned = re.sub(r'[\,\.]', '', cleaned)
        parts = cleaned.strip().split()
        if len(parts) < 2: return full_name
        if len(parts) >= 3 and parts[-2].lower() == 'y':
            surname = f"{parts[-3]} {parts[-2]} {parts[-1]}"
            firstname = " ".join(parts[:-3])
            return f"{surname}, {firstname}"
        surname = parts.pop()
        firstname = " ".join(parts)
        return f"{surname.title()}, {firstname}"

    # --- 1. DATA PREP ---
    CSV_DB = "engine_database.csv"
    if not os.path.exists(CSV_DB):
        print("❌ ERROR: engine_database.csv not found. Run Cell 3 first.")
        return

    pages_to_upload = {}
    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")

    pages_to_upload["engine_database.csv"] = df.to_csv(index=False, encoding='iso-8859-15')
    print("    - [STAGED] engine_database.csv for download")

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar_full = f"""<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Last updated:</strong> {timestamp} &nbsp;|&nbsp; <strong>Total Autosomal matches:</strong> {len(df):,}</div>"""
    stats_bar_html = stats_bar_full

    df['Long_Narrative'] = df.apply(build_narrative, axis=1)
    df['Linked_Tree_Line'] = df.apply(build_linked_lineage, axis=1)

    # --- JSON DATA PREP ---
    ancestor_data = {}
    participant_data = {}
    badge_collections = {"Platinum": set(), "Gold": set(), "Silver": set(), "Bronze": set()}

    if 'Authority_Directory_Label' not in df.columns:
        df['Authority_Directory_Label'] = df['Authority_FirstAncestor_dated']

    # --- ANCESTOR LOOP ---
    for auth_label, group_df in df.groupby('Authority_Directory_Label'):
        match_count = int(len(group_df))
        if match_count < 2: continue

        total_cm = int(group_df['cM'].sum())
        unique_count = int(len(group_df['Tester-Participant-Unmasked'].unique()))
        participants_in_group = group_df['Tester-Participant-Unmasked'].unique()
        top_testers = group_df['Tester-Participant-Unmasked'].value_counts().head(3).to_dict()
        alpha_key = group_df.iloc[0]['Authority_FirstAncestor_alpha']

        # EXTRACT ROOT ID
        root_id = "unknown"
        if 'Ancestral_Path_IDs' in group_df.columns:
            path_str = str(group_df.iloc[0]['Ancestral_Path_IDs'])
            if path_str: root_id = path_str.split(',')[0]

        badge = "Bronze"; integrity = 25; verdict = f"This line is an **Emerging Frontier**. With {match_count} matches, the connection is real but fragile."
        if "Disconnected" in alpha_key or "ZZ_" in alpha_key:
            badge = "Action Required"; integrity = 0; verdict = "**Data Quality Issue.** Matches linked to disconnected profiles. Fix parent links in TNG."
        elif match_count >= 5:
            badge = "Silver"; integrity = 50; verdict = f"**Likely Valid.** Supported by {match_count} matches sharing {total_cm} cM."
        if match_count >= 15 and unique_count >= 3:
            badge = "Gold"; integrity = 80; verdict = f"**Strong Genetic Confirmation.** Anchored by {unique_count} independent tester groups."
        if match_count >= 30 and unique_count >= 10:
            badge = "Platinum"; integrity = 100; verdict = f"**The Platinum Standard.** Biologically confirmed beyond reasonable doubt."

        ancestor_data[alpha_key] = {"name": auth_label, "matches": match_count, "cm": total_cm, "testers": unique_count, "badge": badge, "verdict": verdict, "integrity": integrity, "list_data": top_testers, "root_id": root_id}

        if badge in badge_collections:
            for p in participants_in_group: badge_collections[badge].add(p)

    virtual_groups = {}
    for b_name, p_set in badge_collections.items():
        if len(p_set) > 0: virtual_groups[f"Cohort: {b_name} Standard"] = list(p_set)

    # --- PARTICIPANT LOOP ---
    for p_name, group_df in df.groupby('Tester-Participant-Unmasked'):
        match_count = int(len(group_df))
        if match_count < 2: continue

        total_cm = int(group_df['cM'].sum())
        top_anc = group_df['Authority_FirstAncestor_dated'].mode()[0]
        my_ancestors = group_df['Authority_FirstAncestor_dated'].value_counts().head(3).to_dict()

        badge = "Active Cousin"; integrity = 50; verdict = f"**Active Contributor.** Consistent data linking to {top_anc.split('&')[0]}."
        if match_count >= 15: badge = "Keystone Tester"; integrity = 90; verdict = f"**The Keystone Driver.** Structural pillar connecting disparate branches."

        sort_key = make_participant_key(p_name)

        participant_data[p_name] = {
            "name": p_name,
            "sort_key": sort_key,
            "matches": match_count, "cm": total_cm, "testers": 1, "badge": badge, "verdict": verdict, "integrity": integrity, "list_data": my_ancestors}

    smart_packet_json = json.dumps({"ancestors": ancestor_data, "participants": participant_data, "virtual": virtual_groups})

    # DB for JS Tools (Fix NaNs)
    if 'Search_Inventory_IDs' not in df.columns: df['Search_Inventory_IDs'] = ""
    if 'Search_Inventory_Names' not in df.columns: df['Search_Inventory_Names'] = ""

    df['Search_Inventory_IDs'] = df['Search_Inventory_IDs'].fillna("")
    df['Search_Inventory_Names'] = df['Search_Inventory_Names'].fillna("")

    proof_db_json = df[['Authority_Directory_Label','Tester-Participant-Unmasked','cM','ID#','Yates DNA Ancestral Line', 'Search_Inventory_IDs', 'Search_Inventory_Names']].rename(columns={
        'Authority_Directory_Label':'ancestor',
        'Tester-Participant-Unmasked':'participant',
        'cM':'cm',
        'ID#':'id',
        'Yates DNA Ancestral Line':'lineage',
        'Search_Inventory_IDs':'search_ids',
        'Search_Inventory_Names':'search_names'
    }).to_json(orient='records')

    # --- 2. GENERATE PAGES ---

    js_tools = """
    function getSortableName(fullname) {
        if (!fullname) return "zzz";
        let clean = fullname.replace(/\\b(jr\\.?|sr\\.?|iii|iv|ii|esq\\.?|m\\.d\\.?|ph\\.d\\.?)\\b/gi, "");
        clean = clean.replace(/[\\,\\.]/g, "");
        clean = clean.split(/\\bnee\\b|\\bnée\\b/i)[0];
        let parts = clean.trim().split(/\\s+/);
        if (parts.length >= 3 && parts[parts.length-2].toLowerCase() === 'y') {
            return parts[parts.length-3].toLowerCase() + " " + clean.toLowerCase();
        }
        let surname = parts.length > 0 ? parts[parts.length - 1].toLowerCase() : "zzz";
        return surname.toLowerCase() + " " + clean.toLowerCase();
    }
    """

    # 1. Buster
    buster_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates Brick Wall Buster</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.dashboard{{max-width:900px;margin:0 auto;background:white;padding:30px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1)}}.buster-header{{background:#f57f17;color:white;padding:20px;border-radius:8px 8px 0 0;margin:-30px -30px 20px -30px;text-align:center}}.control-panel{{background:#fff3e0;padding:20px;border-radius:8px;border:1px solid #ffcc80;margin-bottom:20px}}.tabs{{display:flex;gap:10px;margin-bottom:15px;border-bottom:2px solid #ddd;}}.tab{{padding:10px 20px;cursor:pointer;background:#eee;border-radius:5px 5px 0 0;font-weight:bold;flex:1;text-align:center;}}.tab.active{{background:#f57f17;color:white;}}select{{width:100%;padding:12px;font-size:16px;border:1px solid #f57c00;border-radius:4px;margin-top:5px}}button{{background:#e65100;color:white;border:none;padding:12px 25px;font-weight:bold;border-radius:4px;cursor:pointer;margin-top:10px;font-size:16px}}button:hover{{background:#ef6c00}}.results-box{{display:none;margin-top:30px}}.finding-box{{background:#e8eaf6;border-left:5px solid #3f51b5;padding:15px;margin-bottom:15px}}.conclusion-box{{background:#e0f2f1;border-left:5px solid #009688;padding:15px;margin-bottom:15px}}.speculation-box{{background:#ffebee;border-left:5px solid #e91e63;padding:15px;margin-bottom:15px}}.checkbox-list{{height:250px;overflow-y:scroll;border:1px solid #f57c00;background:white;border-radius:4px;padding:10px;}}.checkbox-item{{display:block;margin-bottom:5px;padding:5px;border-bottom:1px solid #eee;}}.checkbox-item:hover{{background-color:#ffe0b2;}}.checkbox-item input{{margin-right:10px;transform:scale(1.2);}}.tooltip{{cursor:pointer;font-size:1.2em;margin-left:10px;}}.tooltip:hover{{opacity:0.7;}}.info-bubble{{display:none;background:#e1f5fe;border:1px solid #81d4fa;padding:15px;border-radius:8px;margin-top:10px;color:#01579b;font-size:0.95em;}}
    table{{width:100%;border-collapse:collapse;margin-top:20px}}th{{background:#004d40;color:white;padding:12px;text-align:left}}td{{padding:10px;border-bottom:1px solid #eee}}tr:hover{{background-color:#f1f8e9}}
    </style></head><body><div class="wrap"><div class="dashboard"><div class="buster-header"><h1>🧱 Brick Wall Buster</h1><p>Predictive Forensic Analysis for Stalled Lineages</p></div>{stats_bar_html}{NAV_HTML}{SITE_INFO}<div class="tabs"><div class="tab active" onclick="setMode('ancestor')">1. Define by Ancestor</div><div class="tab" onclick="setMode('custom')">2. Define by Participants</div></div><div class="control-panel"><div id="panel-ancestor"><label style="font-weight:bold;color:#bf360c;">Select Your "End of Line" Ancestor:</label><select id="wallSelect"><option value="">-- Choose the ancestor you are stuck on --</option></select></div><div id="panel-custom" style="display:none;"><label style="font-weight:bold;color:#bf360c;">Select Testers (Scroll & Check): <span class="tooltip" onclick="toggleInfo()" title="Click to learn about Forensic Handshake Strategy">❓</span></label><div id="info-bubble" class="info-bubble"><strong>The Forensic Handshake Strategy:</strong><br>1. <strong>Isolate:</strong> We identify your specific group (the "Cluster").<br>2. <strong>Corroborate:</strong> We look for independent matches that converge on the same ancestral line.<br>3. <strong>Verify:</strong> A "Handshake" occurs when multiple testers confirm the same relationship, ruling out coincidence.</div><div id="testerList" class="checkbox-list"></div></div><button onclick="runAnalysis()">🔨 Bust This Wall</button></div><div id="results" class="results-box"><div class="no-print" style="float:right;"><button onclick="window.print()" style="background:#455a64;font-size:14px;">🖨️ Print Official Report</button></div><h2 style="border-bottom:2px solid #ddd;padding-bottom:10px;margin-top:0;">Forensic Report</h2><div class="finding-box"><strong>🔍 Findings:</strong> <span id="txt-finding"></span></div><div class="conclusion-box"><strong>💡 Study Context:</strong> <span id="txt-conclusion"></span></div><div class="speculation-box"><strong>🚀 Forensic Handshake Prediction:</strong> <span id="txt-speculation"></span></div><div id="bridge-alert" style="display:none;background:#c8e6c9;border:1px solid #4caf50;padding:15px;margin-top:15px;color:#2e7d32;"></div><h3 style="color:#006064;border-bottom:2px solid #ddd;margin-top:30px;">Cluster Register</h3><div id="cluster-table-div"></div></div></div></div><script>const DATA={smart_packet_json};const FULL_DB={proof_db_json};let MODE='ancestor';
    {js_tools}
    function toggleInfo(){{const b=document.getElementById('info-bubble');b.style.display=(b.style.display==='block')?'none':'block';}}
    const ancSel=document.getElementById('wallSelect');
    Object.keys(DATA.ancestors).sort((a,b)=>DATA.ancestors[b].name.localeCompare(DATA.ancestors[a].name)).forEach(k=>{{const o=document.createElement('option');o.value=k;o.innerText=DATA.ancestors[k].name;ancSel.appendChild(o);}});
    const testListDiv=document.getElementById('testerList');
    const allTesters=Object.keys(DATA.participants);
    allTesters.sort((a,b)=> DATA.participants[a].sort_key.localeCompare(DATA.participants[b].sort_key));
    let listHTML=""; allTesters.forEach(t=>{{\n        listHTML+=`<label class="checkbox-item"><input type="checkbox" value="${{t}}"> ${{t}}</label>`;\n    }});
    testListDiv.innerHTML=listHTML;
    function setMode(m){{MODE=m;document.querySelectorAll('.tab').forEach(t=>t.classList.remove('active'));event.target.classList.add('active');document.getElementById('panel-ancestor').style.display=(m==='ancestor')?'block':'none';document.getElementById('panel-custom').style.display=(m==='custom')?'block':'none';document.getElementById('results').style.display='none';}}
    function runAnalysis(){{
        let targetName="",clusterCount=0,clusterMembers=[],clusterMatches=0;
        if(MODE==='custom'){{const checkedBoxes=document.querySelectorAll('#testerList input:checked');if(checkedBoxes.length===0)return alert("Select at least one tester.");clusterMembers=Array.from(checkedBoxes).map(cb=>cb.value);targetName="Custom Group ("+clusterMembers.length+" Testers)";}} else {{const key=ancSel.value;if(!key)return;const d=DATA.ancestors[key];targetName=d.name;clusterMembers=Object.keys(d.list_data);}}
        clusterCount=clusterMembers.length;const groupRows=FULL_DB.filter(r=>clusterMembers.includes(r.participant));clusterMatches=groupRows.length;groupRows.sort((a,b)=> b.cm - a.cm);let tbl = '<table class="dataframe"><thead><tr><th>Participant</th><th>cM</th><th>Lineage</th></tr></thead><tbody>';groupRows.forEach(r => {{tbl += `<tr><td>${{r.participant}}</td><td>${{r.cm}}</td><td style="font-size:0.9em;color:#555;">${{r.lineage}}</td></tr>`;}});tbl += '</tbody></table>';document.getElementById('cluster-table-div').innerHTML = tbl;const allMatches=FULL_DB.length,outsideMatches=allMatches-clusterMatches,pct=((clusterMatches/allMatches)*100).toFixed(2);let bridgeHTML="";const clusterRows=FULL_DB.filter(r=>clusterMembers.includes(r.participant));const bridges=clusterRows.filter(r=>r.ancestor!==targetName&&r.ancestor!==null);if(bridges.length>0){{const uniqueBridges=[...new Set(bridges.map(b=>b.participant+" -> "+b.ancestor))];if(uniqueBridges.length>0){{bridgeHTML=`<strong>🌉 BRIDGE DETECTED (Forensic Handshake Verified):</strong><br>A "Bridge" is a participant in your cluster who matches YOU but is <em>also</em> confirmed to belong to a known lineage in the database. We found <strong>${{uniqueBridges.length}}</strong> specific bridge connections.<br><br>These testers act as the "Forensic Handshake" linking your group to the following families:<br><ul>`;uniqueBridges.slice(0,5).forEach(b=>bridgeHTML+=`<li>${{b}}</li>`);bridgeHTML+=`</ul>`;document.getElementById('bridge-alert').innerHTML=bridgeHTML;document.getElementById('bridge-alert').style.display='block';}}else{{document.getElementById('bridge-alert').style.display='none';}}}}else{{document.getElementById('bridge-alert').style.display='none';}}const memStr=(clusterMembers.length>5)?clusterMembers.slice(0,5).join(", ")+"...":clusterMembers.join(", ");let findHTML=`<ul style="margin-top:0;padding-left:20px;"><li><strong>Target Group:</strong> ${{targetName}}</li><li><strong>Participants:</strong> ${{clusterCount}} (${{memStr}})</li><li><strong>Data Volume:</strong> ${{clusterMatches}} confirmed matches within this group.</li></ul>`;document.getElementById('txt-finding').innerHTML=findHTML;document.getElementById('txt-conclusion').innerHTML=`This cluster represents <strong>${{pct}}%</strong> of the total Yates database. There are <strong>${{outsideMatches.toLocaleString()}} other matches</strong> in the study.`;let otherGroups=[];for(const[k,v]of Object.entries(DATA.ancestors)){{if(v.name!==targetName&&v.matches>=5)otherGroups.push(v);}}otherGroups.sort((a,b)=>b.matches-a.matches);const top3=otherGroups.slice(0,3);let html="";if(top3.length===0){{html+="<p>No strong signals found. The database currently lacks a 'Proven Line' (5+ matches) that connects to your group.</p>";}}else{{html+=`<p><strong>Methodology: The Forensic Handshake.</strong> We analyzed your cluster for "Handshakes"—patterns where multiple independent matches converge on the same ancestor to confirm a biological link. These are your most probable lineages:</p><ul>`;top3.forEach(g=>{{html+=`<li><strong>${{g.name}}</strong>: Validated by <strong>${{g.matches}} Handshake Connections</strong> (shared match instances) within your cluster. (Probability: <span style="color:green">High</span>)</li>`;}});html+=`</ul>`;}}document.getElementById('txt-speculation').innerHTML=html;document.getElementById('results').style.display='block';}}</script></body></html>"""
    pages_to_upload["brick_wall_buster.shtml"] = buster_html
    print("    - [GENERATED] Brick Wall Buster (v93)")

    # 2. Lineage Proof Engine (DICTIONARY UPGRADE)
    proof_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Lineage Proof Engine</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.proof-card{{background:white;max-width:1000px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:30px}}.badge-large{{font-size:1.2em;padding:10px 20px;border-radius:30px;font-weight:bold;color:white;display:inline-block;margin-bottom:20px}}.badge-platinum{{background:#e5e4e2;color:#333;border:2px solid #ccc}}.badge-gold{{background:#ffd700;color:#b45f06}}.badge-silver{{background:#c0c0c0;color:#333}}.badge-bronze{{background:#cd7f32}}.badge-descendant{{background:#00bcd4;color:white;}}.stats-grid{{display:grid;grid-template-columns:repeat(auto-fit,minmax(150px,1fr));gap:15px;margin-bottom:30px}}.stat-box{{background:#f8f9fa;padding:15px;border-radius:8px;text-align:center;border:1px solid #ddd}}.stat-val{{font-size:1.8em;font-weight:bold;color:#006064}}.stat-lbl{{color:#666;font-size:0.9em;text-transform:uppercase}}table{{width:100%;border-collapse:collapse;margin-top:20px}}th{{background:#004d40;color:white;padding:12px;text-align:left}}td{{padding:10px;border-bottom:1px solid #eee}}tr:hover{{background-color:#f1f8e9}}select{{padding:10px;font-size:16px;width:100%;max-width:600px;margin-bottom:20px;border:1px solid #006064;border-radius:4px}}
    .search-box{{display:flex;gap:10px;max-width:600px;margin-bottom:20px;}} input{{padding:10px;font-size:16px;border:1px solid #ccc;border-radius:4px;flex:1;}} button{{background:#00838f;color:white;border:none;padding:0 20px;border-radius:4px;cursor:pointer;font-weight:bold;}} button:hover{{background:#006064}}
    </style></head><body><div class="wrap"><h1 class="centerline">🧬 Lineage Proof Engine</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div class="proof-card"><h3 style="color:#006064;margin-top:0;">Verify an Ancestral Line</h3><p>Select a couple to audit, or use the <strong>Ancestor Dictionary</strong> to find a specific ancestor hidden in the sub-branches.</p>

    <label style="font-weight:bold;color:#555;display:block;margin-bottom:5px;">Option 1: Select Verified Ancestor (Z-A)</label>
    <select id="proofSelect" onchange="runProof()"><option value="">-- Select Ancestor --</option></select>

    <label style="font-weight:bold;color:#555;display:block;margin-bottom:5px;margin-top:15px;">Option 2: Ancestor Dictionary (Search by Name or ID)</label>
    <div style="margin-bottom:10px;font-size:0.9em;color:#666;">Type a name like "Jane" or "Yates" to see who is actually in the database.</div>
    <div class="search-box">
        <input type="text" id="dictInput" list="ancestorList" placeholder="Start typing ancestor name..." onchange="runDictSearch()">
        <datalist id="ancestorList"></datalist>
        <button onclick="runDictSearch()">Go</button>
    </div>

    <div id="proof-result" style="display:none;"><div style="text-align:center"><span id="p-badge" class="badge-large"></span></div><div style="text-align:center;margin-bottom:15px;color:#004d40;font-size:1.2em;font-weight:bold;" id="p-title"></div><div class="stats-grid"><div class="stat-box"><div class="stat-val" id="p-matches">0</div><div class="stat-lbl">Matches</div></div><div class="stat-box"><div class="stat-val" id="p-testers">0</div><div class="stat-lbl">Unique Testers</div></div><div class="stat-box"><div class="stat-val" id="p-cm">0</div><div class="stat-lbl">Total cM</div></div><div class="stat-box"><div class="stat-val" id="p-integrity">0%</div><div class="stat-lbl">Integrity</div></div></div><div style="background:#e0f2f1;padding:15px;border-left:5px solid #004d40;margin-bottom:20px;font-style:italic;" id="p-verdict"></div><h4 style="border-bottom:2px solid #ddd;padding-bottom:10px;">Evidence Manifest</h4><div style="max-height:500px;overflow-y:auto;"><table id="evidence-table"><thead><tr><th>Participant</th><th>cM</th><th>Lineage Path</th></tr></thead><tbody></tbody></table></div></div></div></div><script>const DATA={smart_packet_json};const DB={proof_db_json};
    const sel=document.getElementById('proofSelect');
    Object.keys(DATA.ancestors).sort((a,b)=>DATA.ancestors[b].name.localeCompare(DATA.ancestors[a].name)).forEach(k=>{{const o=document.createElement('option');o.value=k;o.innerText=DATA.ancestors[k].name;sel.appendChild(o);}});

    // --- BUILD DICTIONARY (ROBUST V122) ---
    const ancestorMap = new Map();
    DB.forEach(r => {{
        if(!r.search_names || !r.search_ids) return;
        const names = r.search_names.split('|');
        const ids = r.search_ids.split(',');

        // Strict Index Alignment
        for(let i=0; i<Math.min(names.length, ids.length); i++){{
            const cleanID = ids[i].replace(/[^0-9]/g, '');
            if(!cleanID) continue;
            const entryKey = cleanID;
            if(!ancestorMap.has(entryKey)) {{
                ancestorMap.set(entryKey, {{ name: names[i], id: cleanID, count: 0 }});
            }}
            ancestorMap.get(entryKey).count++;
        }}
    }});

    const dataList = document.getElementById('ancestorList');
    const sortedAncs = Array.from(ancestorMap.values()).sort((a,b) => a.name.localeCompare(b.name));

    sortedAncs.forEach(a => {{
        const opt = document.createElement('option');
        opt.value = `${{a.name}} (I${{a.id}})`;
        opt.label = `${{a.count}} Matches`;
        dataList.appendChild(opt);
    }});

    function runProof(){{const key=sel.value;if(!key){{document.getElementById('proof-result').style.display='none';return;}}const d=DATA.ancestors[key];document.getElementById('proof-result').style.display='block';
    renderStats(d.badge, d.name, d.matches, d.testers, d.cm, d.integrity, d.verdict, DB.filter(r=>r.ancestor===d.name));}}

    function runDictSearch(){{
        const rawInput = document.getElementById('dictInput').value;
        if(!rawInput) return;

        // Extract ID
        const match = rawInput.match(/\(I(\d+)\)/);
        let searchID = "";
        if(match) {{
            searchID = match[1];
        }} else {{
            searchID = rawInput.replace(/[^0-9]/g, '');
        }}

        if(!searchID) return alert("Please select an ancestor from the list or enter a numeric ID.");

        // SEARCH IN ROBUST IDS
        const subMatches = DB.filter(r => {{
            if(!r.search_ids) return false;
            return r.search_ids.split(',').some(token => token.replace(/[^0-9]/g, '') === searchID);
        }});

        if(subMatches.length === 0) return alert("No matches found for ID: " + searchID);

        // Stats
        const matchCount = subMatches.length;
        const cmTotal = subMatches.reduce((acc, curr) => acc + (curr.cm || 0), 0);
        const uniqueTesters = new Set(subMatches.map(r => r.participant)).size;
        let integ = Math.min(100, (matchCount * 2) + (uniqueTesters * 10));

        renderStats("Dictionary Result", "Focus: Ancestor I" + searchID, matchCount, uniqueTesters, cmTotal, integ, "<strong>Deep Probe Analysis:</strong> Showing all descendants of ID I" + searchID + " (including spouses).", subMatches);
        document.getElementById('proof-result').style.display='block';
    }}

    function renderStats(badgeTxt, titleTxt, matches, testers, cm, integrity, verdict, rows){{
        const b = document.getElementById('p-badge');
        b.className = 'badge-large badge-' + badgeTxt.toLowerCase().split(' ')[0];
        if(badgeTxt === 'Dictionary Result') b.style.backgroundColor = '#673ab7';
        b.innerText = badgeTxt;
        document.getElementById('p-title').innerText = titleTxt;
        document.getElementById('p-matches').innerText = matches;
        document.getElementById('p-testers').innerText = testers;
        document.getElementById('p-cm').innerText = cm.toLocaleString();
        document.getElementById('p-integrity').innerText = integrity+'%';
        document.getElementById('p-verdict').innerHTML = verdict;
        rows.sort((a,b)=>b.cm-a.cm);
        const tbody=document.querySelector('#evidence-table tbody');tbody.innerHTML='';
        rows.forEach(m=>{{const tr=document.createElement('tr');tr.innerHTML=`<td>${{m.participant}}</td><td>${{m.cm}}</td><td style="font-size:0.9em;color:#555;">${{m.lineage}}</td>`;tbody.appendChild(tr);}});
    }}
    </script></body></html>"""
    pages_to_upload["lineage_proof.html"] = proof_html
    print("    - [GENERATED] Lineage Proof Engine")

    # 3. Dossier
    dossier_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates DNA Forensic Dossier</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.dossier-card{{background:white;max-width:900px;margin:20px auto;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);overflow:hidden;position:relative}}.dossier-header{{background:#006064;color:white;padding:25px;text-align:center}}.dossier-body{{padding:30px}}.badge{{display:inline-block;padding:8px 15px;border-radius:20px;font-weight:bold;color:white;font-size:0.9em;margin-bottom:15px;text-transform:uppercase}}.badge-platinum, .badge-keystone{{background:#e5e4e2;color:#333;border:2px solid #ccc}}.badge-gold{{background:#ffd700;color:#b45f06}}.badge-silver, .badge-active{{background:#c0c0c0;color:#333}}.badge-bronze, .badge-new{{background:#cd7f32}}.metric-grid{{display:grid;grid-template-columns:1fr 1fr;gap:20px;margin-bottom:30px}}.metric-box{{background:#f8f9fa;padding:20px;border-radius:8px;text-align:center;border:1px solid #e9ecef}}.metric-val{{font-size:2em;font-weight:bold;color:#004d40}}.metric-lbl{{color:#666;font-size:0.9em;text-transform:uppercase}}.progress-container{{margin:20px 0}}.progress-bar{{background:#e9ecef;height:10px;border-radius:5px;overflow:hidden}}.progress-fill{{height:100%;background:#00838f;width:0%;transition:width 1s ease}}.verdict-box{{background:#e0f2f1;border-left:5px solid #004d40;padding:20px;margin:20px 0;font-family:'Georgia',serif;font-style:italic;color:#004d40;line-height:1.5}}.contributors{{margin-top:20px;border-top:1px solid #eee;padding-top:20px}}.contributor-item{{display:flex;justify-content:space-between;padding:8px 0;border-bottom:1px solid #f1f1f1}}select{{padding:10px;font-size:16px;width:100%;max-width:500px;margin:20px auto;display:block}}.switch{{text-align:center;margin:20px}}.switch label{{margin:0 15px;font-weight:bold;cursor:pointer;color:#006064}}.action-btn{{padding:10px 20px;background:#00838f;color:white;border:none;border-radius:4px;cursor:pointer;font-size:16px;margin:0 5px}}.action-btn:hover{{background:#006064}}#composite-container{{background:white;max-width:900px;margin:20px auto;padding:20px;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);display:none;}}.comp-table{{width:100%;border-collapse:collapse;}}.comp-table th{{background:#004d40;color:white;padding:12px;text-align:left;}}.comp-table td{{padding:10px;border-bottom:1px solid #eee;}}.footer-total{{margin-top:15px;text-align:right;font-weight:bold;color:#004d40;border-top:2px solid #eee;padding-top:10px;}}</style></head><body><div class="wrap"><h1 class="centerline">Yates DNA Forensic Dossier (v80)</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div class="switch"><label><input type="radio" name="mode" value="ancestor" checked onchange="switchMode()"> Search by Ancestor</label><label><input type="radio" name="mode" value="participant" onchange="switchMode()"> Search by Participant</label></div><div style="text-align:center;margin:30px;"><select id="dossierSelect"><option value="">-- Select --</option></select><div style="margin-top:15px;"><button class="action-btn" onclick="addReport()">Add Report</button><button class="action-btn" style="background:#d32f2f;" onclick="clearReports()">Clear All</button></div></div><div id="composite-container"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">Comparison Dashboard</h2><table class="comp-table"><thead><tr><th>Name</th><th>Role/Badge</th><th>Matches</th><th>Total cM</th><th>Integrity</th></tr></thead><tbody id="comp-body"></tbody></table><div id="comp-footer" class="footer-total"></div></div><div id="report-stack"></div></div><script>const DATA={smart_packet_json};let currentMode = 'ancestor';let compTotalMatches = 0; let compTotalCM = 0; let compCount = 0;
    {js_tools}
    function switchMode(){{currentMode = document.querySelector('input[name="mode"]:checked').value;populateDropdown();clearReports();}}
    function populateDropdown(){{const sel = document.getElementById('dossierSelect');sel.innerHTML = '<option value="">-- Select --</option>';const source = (currentMode === 'ancestor') ? DATA.ancestors : DATA.participants;const sortedKeys = Object.keys(source).sort((a,b) => {{
        if(currentMode === 'ancestor') return source[b].name.localeCompare(source[a].name);
        return source[a].sort_key.localeCompare(source[b].sort_key);
    }});
    sortedKeys.forEach(key => {{ const opt = document.createElement('option'); opt.value = key; opt.innerText = (currentMode === 'participant') ? source[key].name : source[key].name; sel.appendChild(opt); }});}}
    function clearReports(){{document.getElementById('report-stack').innerHTML = '';document.getElementById('comp-body').innerHTML = '';document.getElementById('composite-container').style.display = 'none';document.getElementById('dossierSelect').selectedIndex = 0; compTotalMatches=0; compTotalCM=0; compCount=0;}}
    function addReport(){{const key = document.getElementById('dossierSelect').value;if(!key) return;const d = (currentMode === 'ancestor') ? DATA.ancestors[key] : DATA.participants[key];document.getElementById('composite-container').style.display = 'block';compTotalMatches += d.matches; compTotalCM += d.cm; compCount++;const tr = document.createElement('tr');tr.innerHTML = `<td><b>${{d.name}}</b></td><td>${{d.badge}}</td><td>${{d.matches}}</td><td>${{d.cm.toLocaleString()}}</td><td>${{d.integrity}}%</td>`;document.getElementById('comp-body').appendChild(tr);document.getElementById('comp-footer').innerText = `Composite Evidence: ${{compCount}} Profiles, ${{compTotalMatches.toLocaleString()}} Matches, ${{compTotalCM.toLocaleString()}} cM`;const badgeClass = d.badge.toLowerCase().split(' ')[0];const lblTesters = (currentMode==='ancestor') ? 'Unique Testers' : 'Study Rank';const listTitle = (currentMode === 'ancestor') ? 'Top Contributors' : 'Ancestral Connections';let listHTML = '';for (const [name, count] of Object.entries(d.list_data)) {{listHTML += `<div class="contributor-item"><span>${{name}}</span><span>${{count}} matches</span></div>`;}}const html = `<div class="dossier-card" style="display:block; animation: fadeIn 0.5s;"><div class="dossier-header"><h2 style="margin:0">${{d.name}}</h2><div style="margin-top:10px;opacity:0.9">${{currentMode.toUpperCase()}} REPORT</div></div><div class="dossier-body"><div style="text-align:center"><span class="badge badge-${{badgeClass}}">${{d.badge}}</span></div><div class="verdict-box"><strong>Forensic Analysis:</strong><br>${{d.verdict}}</div><div class="metric-grid"><div class="metric-box"><div class="metric-val">${{d.matches}}</div><div class="metric-lbl">Matches</div></div><div class="metric-box"><div class="metric-val">${{d.testers}}</div><div class="metric-lbl">${{lblTesters}}</div></div><div class="metric-box"><div class="metric-val">${{d.cm.toLocaleString()}}</div><div class="metric-lbl">Total cM</div></div><div class="metric-box"><div class="metric-val">${{d.integrity}}%</div><div class="metric-lbl">Integrity Score</div></div></div><div class="progress-container"><div style="display:flex;justify-content:space-between;margin-bottom:5px;font-size:0.9em;font-weight:bold;"><span>Score</span><span>${{d.integrity}}/100</span></div><div class="progress-bar"><div class="progress-fill" style="width:${{d.integrity}}%"></div></div></div><div class="contributors"><h3>${{listTitle}}</h3><div>${{listHTML}}</div></div></div></div>`;document.getElementById('report-stack').insertAdjacentHTML('afterbegin', html);}}populateDropdown();</script></body></html>"""
    pages_to_upload["dna_dossier.html"] = dossier_html
    print("    - [GENERATED] Forensic Dossier")

    # Guide
    contents_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates Study User Guide</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f4f7f6;padding:20px}}.guide-grid{{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto}}.guide-card{{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s}}.guide-card:hover{{transform:translateY(-5px)}}.card-title{{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}}.card-why{{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}}.card-what{{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}}.card-btn{{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}}.card-btn:hover{{background:#006064}}</style></head><body><div class="wrap"><h1 class="centerline">Welcome to the Yates DNA Study Portal</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div style="text-align:center;max-width:800px;margin:20px auto;color:#444;font-size:1.1em;">This site transforms raw DNA data into forensic genealogical evidence. Use the tools below to explore your heritage, verify ancestors, and analyze the strength of your genetic connections.</div>
    <div class="guide-grid">
        <div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all 1,700+ DNA matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div>
        <div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically "saturated" and proven by multiple testers.</div><a href="dna_network.shtml" class="card-btn">View Network</a></div>
        <div class="guide-card"><h2 class="card-title">3. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed by independent cousins.</div><a href="lineage_proof.html" class="card-btn">Run Proof</a></div>
        <div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">4. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">To break through a dead end. This predictive engine uses "Negative Space Analysis" to suggest which proven family line you likely belong to based on who you match.</div><a href="brick_wall_buster.shtml" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div>
        <div class="guide-card"><h2 class="card-title">5. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">To get your "Scorecard." Generate a one-page forensic report on yourself or an ancestor, grading the strength of the evidence.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div>
        <div class="guide-card"><h2 class="card-title">6. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. A high-level audit showing participant statistics, masked IDs, and total study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div>
        <div class="guide-card"><h2 class="card-title">7. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">To understand the terms. Definitions for "Keystone Tester," "Platinum Standard," and other forensic terms used in this study.</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div>
        <div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">8. Corrections & Mysteries</h2><div class="card-why">See Something? Say Something.</div><div class="card-what">We know we aren't 100% right, nor 100% wrong. Genealogy is a collaboration. If you can solve a mystery or correct a detail, tell us. <br><br><strong>Important:</strong> Please copy/paste the <strong>ID# (e.g., I1234)</strong> so we can fix the correct person.</div><a href="mailto:yates@one-name.org?subject=Yates DNA Study: Correction / Mystery&body=I have a correction or potential update regarding Person ID: [PASTE ID HERE]. Here are the details:" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div>
    </div></div></body></html>"""
    pages_to_upload["contents.shtml"] = contents_html
    print("    - [GENERATED] Contents Guide")

    # Subscribe & Theory & Share
    pages_to_upload["subscribe.shtml"] = make_page("Subscribe", SUBSCRIBE_CONTENT, len(df), "subscribe", stats_bar=stats_bar_html)
    pages_to_upload["share_dna.shtml"] = make_page("Share Your DNA", SHARE_CONTENT, len(df), "share", stats_bar=stats_bar_html)
    pages_to_upload["dna_theory_of_the_case.htm"] = make_page("The Yates DNA Strategy", THEORY_CONTENT, len(df), "theory", stats_bar=stats_bar_html)

    # Admin Hub
    part_stats = df.groupby('Tester-Participant-Unmasked').agg({'Tester-Participant-MASKED': 'first', 'ID#': 'count'}).reset_index().rename(columns={'ID#': 'Match_Count'})
    part_stats['Sort_Key'] = part_stats['Tester-Participant-Unmasked'].apply(get_sortable_surname)
    part_stats = part_stats.sort_values(['Sort_Key', 'Tester-Participant-Unmasked'], ascending=[True, True])
    total_matches = part_stats['Match_Count'].sum(); part_stats['Share_Pct'] = (part_stats['Match_Count'] / total_matches) * 100
    admin_rows = []
    for _, r in part_stats.iterrows():
        name = r['Tester-Participant-Unmasked']
        full_sort_val = get_sortable_surname(name)
        display_name = format_last_first(name) # V86: Flip Name in Admin Table
        row = f"<tr><td>{r['Tester-Participant-MASKED']}</td><td data-sort='{full_sort_val}'><b>{display_name}</b></td><td>{r['Match_Count']}</td><td>{r['Share_Pct']:.2f}%</td></tr>"
        admin_rows.append(row)

    # V86: Admin Buttons
    admin_buttons = """<div style="text-align:center;margin:20px 0;"><a href="admin_singletons.shtml" class="action-btn" style="background:#fbc02d;color:#333;margin-right:10px;">🔍 View Singleton Lines (Housekeeping)</a><a href="engine_database.csv" class="action-btn" style="background:#455a64;">⬇️ Download Raw Data (CSV)</a></div>"""

    pages_to_upload["research_admin.html"] = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates Research Admin Hub</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.dashboard-grid{{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:20px;margin:30px auto;max-width:1200px}}.dash-card{{background:white;padding:20px;border-radius:8px;text-align:center;box-shadow:0 4px 6px rgba(0,0,0,0.1);transition:transform 0.2s;text-decoration:none;color:#333;border:1px solid #ddd}}.dash-card:hover{{transform:translateY(-5px);border-color:#006064;background:#e0f7fa}}.dash-icon{{font-size:40px;margin-bottom:10px;display:block}}.dash-title{{font-weight:bold;font-size:1.1em;color:#006064}}.audit-table-wrapper{{background:white;padding:25px;border-radius:8px;box-shadow:0 4px 6px rgba(0,0,0,0.1);max-width:1200px;margin:0 auto}}.audit-table{{width:100%;border-collapse:collapse}}.audit-table th{{background:#004d40;color:white;padding:12px;text-align:left;position:sticky;top:0}}.audit-table td{{padding:10px;border-bottom:1px solid #eee}}.audit-table tr:hover{{background-color:#f5f5f5}}.total-row{{background:#e0f2f1;font-weight:bold;border-top:2px solid #004d40}}.action-btn{{padding:10px 20px;text-decoration:none;border-radius:4px;font-weight:bold;display:inline-block;}}</style></head><body><div class="wrap"><h1 class="centerline">Yates Research Admin Hub</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div><div class="dashboard-grid"><a href="ons_yates_dna_register.shtml" class="dash-card"><span class="dash-icon">📋</span><span class="dash-title">DNA Register</span></a><a href="dna_network.shtml" class="dash-card"><span class="dash-icon">🕸️</span><span class="dash-title">DNA Network</span></a><a href="lineage_proof.html" class="dash-card"><span class="dash-icon">🧬</span><span class="dash-title">Proof Engine</span></a><a href="dna_dossier.html" class="dash-card"><span class="dash-icon">📁</span><span class="dash-title">Forensic Dossier</span></a><a href="just-trees.shtml" class="dash-card"><span class="dash-icon">🌳</span><span class="dash-title">Trees View</span></a></div><div class="audit-table-wrapper"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">Participant Activity Report (Sorted by Surname)</h2>{admin_buttons}<div style="max-height:600px;overflow-y:auto;"><table class="audit-table sortable"><thead><tr><th>Masked ID</th><th>Unmasked Participant</th><th>Matches</th><th>% Share</th></tr></thead><tbody>{''.join(admin_rows)}</tbody><tfoot><tr class="total-row"><td colspan="2" style="text-align:right;padding-right:20px;">TOTAL DATABASE:</td><td>{total_matches}</td><td>100%</td></tr></tfoot></table></div></div></div><script>{js_tools}</script></body></html>"""
    print("    - [GENERATED] Research Admin Hub (with Download & Singleton Links)")

    # Glossary & Registers
    pages_to_upload["data_glossary.shtml"] = make_page("Data Glossary", GLOSSARY_CONTENT, len(df), "glossary", stats_bar=stats_bar_html)

    # --- FIXED SECTION: PARTICIPANT REGISTER SORTING ---
    df_par = df.copy()
    # SORT BY YOUR NEW COLUMN (3rd column: tester-cojoin-lastfirst)
    if 'tester-cojoin-lastfirst' in df_par.columns:
        df_par.sort_values(by=['tester-cojoin-lastfirst', 'Found Match'], ascending=[True, True], inplace=True)

    df_par.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    pages_to_upload["ons_yates_dna_register_participants.shtml"] = make_page("ONS Yates Study DNA Register", f'<div class="table-scroll-wrapper" style="margin:0 auto;width:90%;">{df_par.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', len(df), "participant", stats_bar=stats_bar_html)

    # 2. ANCESTOR REGISTER (Show 2+ Matches Only)
    match_counts = df['Authority_Directory_Label'].value_counts()
    valid_ancestors = match_counts[match_counts >= 2].index
    df_anc = df[df['Authority_Directory_Label'].isin(valid_ancestors)].copy()

    stats_bar_anc = f"""<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Last updated:</strong> {timestamp} &nbsp;|&nbsp; <strong>Validated Matches (2+):</strong> {len(df_anc):,} <span style="color:#d32f2f;">(Singleton matches hidden)</span></div>"""

    # Sort by the new Directory Label (Last, First)
    df_anc.sort_values(by=['Authority_Directory_Label', 'Yates DNA Ancestral Line'], ascending=[True, True], inplace=True)
    df_anc_disp = df_anc.copy()
    df_anc_disp.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)

    pages_to_upload["ons_yates_dna_register.shtml"] = make_page("ONS Yates Study DNA Register", f'<div class="table-scroll-wrapper" style="margin:0 auto;width:90%;">{df_anc_disp.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', len(df_anc), "ancestor", stats_bar=stats_bar_anc)
    pages_to_upload["yates_ancestor_register.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]

    # 3. SINGLETON REGISTER (Housekeeping View)
    singleton_ancestors = match_counts[match_counts == 1].index
    df_single = df[df['Authority_Directory_Label'].isin(singleton_ancestors)].copy()
    stats_bar_single = f"""<div style="background:#fff3e0;border-top:1px solid #ffcc80;border-bottom:1px solid #ffcc80;font-family:sans-serif;font-size:12px;color:#e65100;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>HOUSEKEEPING VIEW:</strong> Showing {len(df_single):,} singleton matches. These lines need confirmation.</div>"""
    df_single.sort_values(by=['Authority_Directory_Label'], ascending=[True])
    df_single_disp = df_single.copy()
    df_single_disp.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    pages_to_upload["admin_singletons.shtml"] = make_page("Singleton Match Register", f'<div class="table-scroll-wrapper" style="margin:0 auto;width:90%;">{df_single_disp.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', len(df_single), "singleton", stats_bar=stats_bar_single)
    print("    - [GENERATED] Singleton Register (Housekeeping)")

    # Trees
    df_tree = df_anc[['Linked_Tree_Line', 'Authority_Directory_Label']].copy()
    df_tree.rename(columns={'Linked_Tree_Line': 'TEMP'}, inplace=True)

    # Trees sorted Z-A (default)
    df_tree.sort_values(by=['Authority_Directory_Label'], ascending=[False], inplace=True)
    pages_to_upload["just-trees.shtml"] = make_page("Ancestor Register (Trees View)", f'<div class="table-scroll-wrapper" style="margin:0 auto;width:90%;">{df_tree[["TEMP"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table").replace("<th>TEMP</th>", "<th>&nbsp;</th>")}</div>', len(df_anc), "tree_za", stats_bar=stats_bar_anc)

    # Trees sorted A-Z
    df_tree.sort_values(by=['Authority_Directory_Label'], ascending=[True], inplace=True)
    pages_to_upload["just-trees-az.shtml"] = make_page("Ancestor Register (Trees View)", f'<div class="table-scroll-wrapper" style="margin:0 auto;width:90%;">{df_tree[["TEMP"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table").replace("<th>TEMP</th>", "<th>&nbsp;</th>")}</div>', len(df_anc), "tree_az", stats_bar=stats_bar_anc)

    # Network (Sorted by Size/Saturation)
    network_buffer = []
    # Sort groups by count descending (largest first)
    sorted_groups = sorted(df.groupby('Authority_Directory_Label'), key=lambda x: len(x[1]), reverse=True)

    for ancestor_label, group_df in sorted_groups:
        if len(group_df) < 2: continue
        total_cm_g = group_df['cM'].sum(); unique_testers_g = len(group_df['Tester-Participant-Unmasked'].unique())
        analyzer_comment = f"""<div style="background:#fffde7;border-left:6px solid #fbc02d;padding:10px;margin-bottom:15px;font-family:sans-serif;color:#333;font-size:0.95em;"><strong>Collateral Saturation Analysis:</strong> Validated by <b>{unique_testers_g} independent testers</b> representing distinct lines. This cluster aggregates <b>{len(group_df)} matches</b> and <b>{total_cm_g:.0f} cM</b> of shared DNA, providing forensic confirmation of the ancestral couple.</div>"""
        group_df = group_df.sort_values('cM', ascending=False)
        network_buffer.append(f"""<details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">{ancestor_label}</span> <span style="float:right;color:#004d40;font-size:0.9em;">Matches: {len(group_df)} | Total cM: {total_cm_g:.0f}</span></summary><div style="padding:15px;">{analyzer_comment}<table class="dataframe" border="1"><thead><tr style="text-align:left;"><th>Tester</th><th>cM</th><th>Lineage</th></tr></thead><tbody>""")
        for _, r in group_df.iterrows(): network_buffer.append(f"<tr><td>{r['Tester-Participant-Unmasked']}</td><td>{r['cM']}</td><td>{r['Yates DNA Ancestral Line']}</td></tr>")
        network_buffer.append("</tbody></table></div></details>")
    pages_to_upload["dna_network.shtml"] = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Participating DNA Network</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css"><style>summary::-webkit-details-marker{{display:none}}summary{{outline:none}}</style></head><body id="top"><div class="wrap"><h1 class="centerline">Participating DNA Network</h1><div id="nav-slot">{stats_bar_html}{NAV_HTML}</div>{SITE_INFO}<div style="margin:20px auto;width:90%;">{"".join(network_buffer)}</div><script>{js_tools}</script></body></html>"""
    print("    - [GENERATED] DNA Network")

    # --- 3. UPLOAD (Self-Contained) ---
    print("\n[STEP 3] Uploading to Server...")
    if len(pages_to_upload) > 0:
        try:
            # Inline Connection Logic (No external dependency)
            from ftplib import FTP_TLS
            HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
            USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
            PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

            ftps = FTP_TLS()
            ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()

            target_dirs = [f"/{REMOTE_SUBDIR}", f"/public_html/{REMOTE_SUBDIR}", f"/public_html/yates.one-name.net/{REMOTE_SUBDIR}", f"htdocs/{REMOTE_SUBDIR}", REMOTE_SUBDIR]
            found_dir = False
            for d in target_dirs:
                try:
                    ftps.cwd(d)
                    found_dir = True
                    print(f"[SUCCESS] Locked onto correct web directory: {d}")
                    break
                except: pass

            if not found_dir:
                print(f"\n[CRITICAL WARNING] Could not locate the '{REMOTE_SUBDIR}' folder!")
                print(f"STUCK IN: {ftps.pwd()}")
                print("Aborting upload to avoid ghost files.")
            else:
                for fn, content in pages_to_upload.items():
                    with open(fn, "w", encoding="utf-8") as f: f.write(content)
                    with open(fn, "rb") as fh: ftps.storbinary(f"STOR {fn}", fh)
                    print(f"    - Uploaded: {fn}")
                print(f"\n[SUCCESS] All Files Published Live to {ftps.pwd()}")
            ftps.quit()
        except Exception as e: print(f"\n[ERROR] Upload Failed: {e}")
    else: print("[WARN] No content generated to upload.")

print("✅ Cell 4 (Publisher V122) Loaded.")

✅ Cell 4 (Publisher V122) Loaded.


<>:325: SyntaxWarning: invalid escape sequence '\('
<>:325: SyntaxWarning: invalid escape sequence '\('
/tmp/ipython-input-508980715.py:325: SyntaxWarning: invalid escape sequence '\('
  dossier_html = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates DNA Forensic Dossier</title><link rel="stylesheet" href="partials_unified.css"><style>body{{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}}.dossier-card{{background:white;max-width:900px;margin:20px auto;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);overflow:hidden;position:relative}}.dossier-header{{background:#006064;color:white;padding:25px;text-align:center}}.dossier-body{{padding:30px}}.badge{{display:inline-block;padding:8px 15px;border-radius:20px;font-weight:bold;color:white;font-size:0.9em;margin-bottom:15px;text-transform:uppercase}}.badge-platinum, .badge-keystone{{background:#e5e4e2;color:#333;border:2px solid #ccc}}.badge-gold{{background:#ffd700;color:#b45f06}}.badge-si

In [53]:
# @title [CELL 5] MASTER ORCHESTRATOR (Run This Button)
import os, sys
print("="*60)
print("      MASTER ORCHESTRATOR (V81)")
print("      (Running Engine -> Publisher -> Upload)")
print("="*60)

if 'run_engine' not in globals() or 'run_publisher' not in globals():
    print("❌ ERROR: Modules not loaded! Please run Cells 3 and 4 first.")
else:
    print("\n>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...")
    try:
        run_engine()
        print("✅ PHASE 1 COMPLETE.")

        print("\n>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...")
        run_publisher()
        print("✅ PHASE 2 COMPLETE.")

        print("\n" + "="*60)
        print("      🏆 V81 UPDATE SUCCESSFUL")
        print("="*60)
    except Exception as e:
        print(f"\n❌ CRITICAL FAILURE: {e}")

      MASTER ORCHESTRATOR (V81)
      (Running Engine -> Publisher -> Upload)

>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...
      [CELL 3] ENGINE STARTING (V123.2 - DEEP SPACE PROBE)...
    🗑️ Deleted old database (Forcing fresh build)

[STEP 1] Setup...
    👉 Source: yates_study_2025.ged

[STEP 4] Tracing Lineages...

[SUCCESS] Engine V123.2 Complete. Saved 1712 verified matches to engine_database.csv (Deep Space Probe Active).
✅ PHASE 1 COMPLETE.

>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...
      [CELL 4] PUBLISHER STARTING (V122 - ROBUST SEARCH)...
    - [STAGED] engine_database.csv for download
    - [GENERATED] Brick Wall Buster (v93)
    - [GENERATED] Lineage Proof Engine
    - [GENERATED] Forensic Dossier
    - [GENERATED] Contents Guide
    - [GENERATED] Research Admin Hub (with Download & Singleton Links)
    - [GENERATED] Singleton Register (Housekeeping)
    - [GENERATED] DNA Network

[STEP 3] Uploading to Server...
[SUCCESS] Locked onto correct web directory: /ons-study
    -

In [ ]:
# @title [CELL 6] The Time Machine (Archiver + Dropbox Sync)
import zipfile
import os
import pytz
import time
from datetime import datetime
from google.colab import files
from google.colab import userdata

# --- 1. INSTALL DROPBOX (IF MISSING) ---
try:
    import dropbox
    from dropbox.exceptions import AuthError
except ImportError:
    os.system('pip install dropbox')
    import dropbox
    from dropbox.exceptions import AuthError

def run_archiver():
    print("="*60)
    print("      [CELL 6] MANUAL ARCHIVER + DROPBOX SYNC")
    print("="*60)

    # --- 2. CREATE ZIP (SAFE MODE) ---
    # We explicitly exclude .zip to prevent "Zip Bombs"
    extensions = ('.csv', '.shtml', '.html', '.json', '.js', '.css')
    files_to_pack = [f for f in os.listdir('.') if f.lower().endswith(extensions) and "sample_data" not in f]

    if not files_to_pack:
        print("❌ No generated files found! Run the Publisher (Cell 4) first.")
        return

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%Y-%m-%d_%H%M")
    zip_name = f"Yates_Study_Backup_{timestamp}.zip"

    print(f"📦 Compressing {len(files_to_pack)} files into {zip_name}...")
    try:
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
            for file in files_to_pack:
                zf.write(file)
        print(f"    ✅ Archive Created: {zip_name} ({os.path.getsize(zip_name)/1024:.1f} KB)")
    except Exception as e:
        print(f"    ❌ Compression Failed: {e}")
        return

    # --- 3. FTP UPLOAD (BACKUPS FOLDER) ---
    print("\n[STEP 2] Uploading to Web Server (FTP)...")
    try:
        from ftplib import FTP_TLS
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()

        try:
            ftps.cwd("/ons-study/backups")
        except:
            try:
                ftps.mkd("/ons-study/backups")
                ftps.cwd("/ons-study/backups")
            except:
                pass

        with open(zip_name, "rb") as fh:
            ftps.storbinary(f"STOR {zip_name}", fh)
        print(f"    ✅ FTP Success: /ons-study/backups/{zip_name}")
        ftps.quit()
    except Exception as e:
        print(f"    ⚠️ FTP Upload skipped: {e}")

    # --- 4. DROPBOX SYNC (NEW) ---
    print("\n[STEP 3] Syncing to Dropbox...")
    try:
        # Initialize with Refresh Token (Long-term access)
        dbx = dropbox.Dropbox(
            app_key=userdata.get('DBX_APP_KEY'),
            app_secret=userdata.get('DBX_APP_SECRET'),
            oauth2_refresh_token=userdata.get('DBX_REFRESH_TOKEN')
        )

        # Upload the Zip
        target_path = f"/Backups/{zip_name}"
        with open(zip_name, "rb") as f:
            dbx.files_upload(f.read(), target_path, mode=dropbox.files.WriteMode.overwrite)

        print(f"    ✅ Dropbox Success: {target_path}")

    except Exception as e:
        print(f"    ❌ Dropbox Upload Failed: {e}")
        print("       (Check DBX_APP_KEY, DBX_APP_SECRET, DBX_REFRESH_TOKEN in Colab Secrets)")

    # --- 5. LOCAL DOWNLOAD (SAFETY NET) ---
    print("\n[STEP 4] Triggering Local Download...")
    try:
        files.download(zip_name)
    except Exception as e:
        print(f"    ⚠️ Auto-download blocked: {e}")

    print("✅ Archival Process Complete.")

# Run it
run_archiver()

      [CELL 6] MANUAL ARCHIVER + DROPBOX SYNC
📦 Compressing 17 files into Yates_Study_Backup_2026-02-16_2133.zip...
    ✅ Archive Created: Yates_Study_Backup_2026-02-16_2133.zip (744.1 KB)

[STEP 2] Uploading to Web Server (FTP)...
    ✅ FTP Success: /ons-study/backups/Yates_Study_Backup_2026-02-16_2133.zip

[STEP 3] Syncing to Dropbox...
    ✅ Dropbox Success: /Backups/Yates_Study_Backup_2026-02-16_2133.zip

[STEP 4] Triggering Local Download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Archival Process Complete.
